In [ ]:
from pyvis.network import Network
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 불용어 리스트 정의 (한국어 불용어)
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀', '것', '달바', '미스트', '스프레이', '1으로', '1이라', '1에', '너무', '느낌', '잘', '그냥', '많이', 
              '있어서', '같아요', '스프레이가', '미스트가', '미스트는', '약간의', '생각보다', '있으며', '살짝', '같아서', 
              '느낌이', '저는', '진짜', '엄청', '합니다', '더', '안', '느낌은', '효과는', '조금', '분들은', '크림미스트라서',
              '크림미스트라', '크림', '크림이', '미스트라', '미스트라서', '그런지', '못', '이거', '완전', '아벤느', '다', '개', '걸',
              '매우','음','확','전','제','달바는','막','넘','너무너무','특히','근데','취향이','정말','덜','뭔가',
              '저의','굉장히','아니지만','아니고','해야할까요','제품','시',
              '있는데','샀는데','아니라','기분','썼을것같은데','바꿔봤는데','그런','사봤는데','해야할까요','저에게는',
              '하는데','아주','알겠더라구용','되게','나요','나서','나면서','진심','나니까','저한텐','저한테는','구매했어요',]

# 데이터 로드
data = pd.read_excel(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_dalba.xlsx")
data = data.where(pd.notna(data), None)  # 결측치 처리
data = data.reset_index()

# 인덱스 넘버가 3 + 7 * n인 경우 필터링하여 'Aspect2 Mapped Labels' 행 필터링
filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]

# 키워드 설정
aspect_keywords = ["봄"]

# 텍스트 전처리 함수
def preprocess_text(text):
    """텍스트에서 특수문자 제거 및 불용어 필터링"""
    cleaned_text = re.sub(r"[^가-힣0-9\s]", "", text)
    tokens = [word for word in cleaned_text.split() if word not in stopwords]
    return tokens

# 네트워크 그래프 생성을 위한 데이터 수집
nodes_data = set()  # 중복 방지를 위해 set 사용
all_previous_tokens = []

# 키워드와 관련된 단어 연결
for idx, row in filtered_data.iterrows():
    original_idx = data[data['index'] == row['index']].index[0]
    tokenized_text_index = original_idx + 3  # 'Tokenized Text' 행
    sentiment_row_index = tokenized_text_index - 5  # 감정 데이터는 동일 열의 -5행에 위치

    if tokenized_text_index < len(data):

        for col_index in range(2, len(data.columns)):  # 각 열을 순회
            tokenized_text = str(data.iat[tokenized_text_index, col_index])

            # 감정 데이터 가져오기
            try:
                sentiment_value = str(data.iat[sentiment_row_index, col_index]).strip()
            except Exception as e:
                print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
                sentiment_value = None

            # Null 또는 빈 문자열 확인
            if not tokenized_text or tokenized_text.strip() == "":
                continue

            # 키워드 포함 여부 확인
            if not any(keyword in tokenized_text for keyword in aspect_keywords):
                continue

            # 키워드가 포함된 경우 처리
            print(f"[INFO] 키워드 포함 텍스트: {tokenized_text}, 감정: {sentiment_value}")

            # 기준 셀의 왼쪽과 오른쪽 셀 값을 추출
            left_text = (
                str(data.iat[tokenized_text_index, col_index - 1])
                if col_index > 2  # 왼쪽 셀이 존재하는 경우
                else None
            )
            right_text = (
                str(data.iat[tokenized_text_index, col_index + 1])
                if col_index < len(data.columns) - 1  # 오른쪽 셀이 존재하는 경우
                else None
            )

            # 제외할 접미사를 리스트로 관리
            EXCLUDED_SUFFIXES = ("-")
            EXCLUDED_KEYWORDS = ["-"]
            # 왼쪽 셀 추가
            if left_text and left_text.strip():
                stripped_left_text = left_text.strip()  # 공백 제거
                # 접미사 검사
                if any(stripped_left_text.endswith(suffix) for suffix in EXCLUDED_SUFFIXES):
                    print(f"[INFO] 왼쪽 셀이 '{EXCLUDED_SUFFIXES}' 중 하나로 끝나서 제외됨: {stripped_left_text}")

                elif any(keyword in stripped_left_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 왼쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_left_text}")
                else:
                    nodes_data.add((tokenized_text, stripped_left_text, tokenized_text_index, col_index))  # set에 추가
                    all_previous_tokens.append(stripped_left_text)

            # 오른쪽 셀 추가
            if right_text and right_text.strip():
                stripped_right_text = right_text.strip()  # 공백 제거
                # 키워드 포함 여부 확인
                if any(keyword in stripped_right_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 오른쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_right_text}")
                else:
                    nodes_data.add((tokenized_text, right_text.strip(),tokenized_text_index,col_index))  # set에 추가
                    all_previous_tokens.append(right_text.strip())

 

# 빈도수 카운터 생성
if not all_previous_tokens:
    print("[ERROR] 관련된 서술어가 없습니다. 전처리 또는 데이터 로드 로직을 확인하세요.")
else:
    # 감정별로 빈도수를 따로 저장하기 위한 딕셔너리
    sentiment_frequency = {
        "긍정": Counter(),
        "부정": Counter(),
        "중립": Counter()
    }

    # 전체 빈도수 저장
    total_frequency = Counter()

    # 빈도수 계산
    for tokenized_text, related_text, tokenized_text_index, col_index in nodes_data:
        # 감정 값 가져오기
        try:
            sentiment_value = str(data.iat[tokenized_text_index - 5, col_index]).strip()
            # B-긍정, I-긍정 같은 값을 "긍정"으로 통합
            if sentiment_value.startswith("B-") or sentiment_value.startswith("I-"):
                sentiment_value = sentiment_value[2:]
        except Exception as e:
            print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
            sentiment_value = None

        # 유효한 감정 값인지 확인
        if sentiment_value in sentiment_frequency:
            sentiment_frequency[sentiment_value][related_text] += 1
            total_frequency[related_text] += 1  # 전체 빈도수 업데이트



In [ ]:
# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")
net.toggle_physics(True)

# 중심 노드와 연관 노드 간 연결 정보 기록
connected_nodes = {}
center_frequency = Counter()

# 그래프 노드 및 엣지 추가
EXCLUDED_MAIN_KEYWORDS = ["-"]

for main_token, related_token, sentiment_row_index, col_index in nodes_data:
    main_node_key = main_token.strip()  # 중심 노드 키 생성
    print(f"[DEBUG] Processing pair: main_token={main_token}, related_token={related_token}")

    # 유효성 검증 및 불용어 필터링
    if not re.match(r"^[가-힣]+$", main_node_key) or not re.match(r"^[가-힣]+$", related_token):
        print(f"[INFO] main_token 또는 related_token이 유효하지 않음: {main_token}, {related_token}")
        continue
    if main_token in stopwords or related_token in stopwords:
        print(f"[INFO] 불용어로 제외됨: {main_token} 또는 {related_token}")
        continue
    if any(keyword in main_node_key for keyword in EXCLUDED_MAIN_KEYWORDS):
        print(f"[INFO] 중심 노드가 제외 키워드를 포함하여 추가되지 않음: {main_node_key}")
        continue

    # 감정 값 가져오기
    related_token_sentiment = None
    try:
        related_token_sentiment = str(data.iat[sentiment_row_index - 5, col_index]).strip()
        if related_token_sentiment.startswith("B-") or related_token_sentiment.startswith("I-"):
            related_token_sentiment = related_token_sentiment[2:]  # 통합
    except Exception as e:
        print(f"[ERROR] related_token 감정 데이터 가져오기 실패: {e}")
        continue

    # 연관 노드 추가
    related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
    if related_token_frequency >= 1:
        if "O" in related_token_sentiment:
            print(f"[INFO] 연관 노드가 '중립' 또는 'O' 감정으로 추가되지 않음: {related_token}")
            continue

        related_node_key = f"{related_token}_{related_token_sentiment}"
        if related_node_key not in net.get_nodes():
            node_size = min(15 + related_token_frequency * 2, 50)
            related_token_color = {
                "긍정": "#6EC664",
                "중립": "#BDC3C7",
                "부정": "#E74C3C"
            }.get(related_token_sentiment, "black")
            net.add_node(
                related_node_key,
                label=related_token,
                color=related_token_color,
                size=node_size,
                title=f"{related_token} (빈도: {related_token_frequency})"
            )
            print(f"[DEBUG] 추가된 연관 노드: {related_node_key}")

        # 엣지 및 중심 노드 추가
        if main_node_key not in net.get_nodes():
            net.add_node(
                main_node_key,
                label=main_token,
                color={"background": "white", "border": "black"},
                size=25,
                title=f"{main_token} (빈도: 0)"  # 빈도수는 추후 갱신
            )
        # 엣지 추가
        net.add_edge(main_node_key, related_node_key, color=related_token_color)
        print(f"[DEBUG] 엣지 추가됨: {main_node_key} -> {related_node_key}")
        connected_nodes.setdefault(main_node_key, []).append(related_node_key)

# 중심 노드 빈도 계산 (화면에 표시된 연관 노드 기반)
center_frequency.clear()  # 기존 빈도수 초기화
for main_node_key, related_nodes in connected_nodes.items():
    for related_node_key in related_nodes:
        related_token = related_node_key.split("_")[0]
        related_token_sentiment = related_node_key.split("_")[1]
        related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
        center_frequency[main_node_key] += related_token_frequency
        print(f"[DEBUG] 중심 노드 '{main_node_key}'에 '{related_token}'의 빈도 {related_token_frequency} 추가됨")

# 중심 노드 빈도 업데이트 (화면에 표시된 값으로 갱신)
for node in net.nodes:
    if node["id"] in center_frequency:
        node["title"] = f"{node['id']} (빈도: {center_frequency[node['id']]})"
        print(f"[DEBUG] 중심 노드 '{node['id']}'의 빈도 갱신됨: {center_frequency[node['id']]}")

# 디버깅: 최종 중심 노드 빈도수 출력
print("[DEBUG] 최종 중심 노드 빈도수:")
for node, freq in center_frequency.items():
    print(f"{node}: {freq}")

# 네트워크 그래프 시각화
net.show("spring.html")


In [ ]:
import pandas as pd
import re

# 네트워크에 표시된 모든 노드 데이터 가져오기
all_nodes_data = []
for node in net.nodes:
    # 노드 정보 추출
    node_id = node["id"]
    node_label = node.get("label", "")
    node_color = node.get("color", "")
    node_title = node.get("title", "")  # 빈도수 정보 포함 가능

    # 노드 종류 결정
    if "_" in node_id:  # "_"이 포함된 경우 연관 노드
        node_type = "연관 노드"
    else:  # "_"이 없는 경우 중심 노드
        node_type = "중심 노드"

    # 감정 추출 (색상 기반)
    if node_color == "#6EC664":  # 연한 녹색
        node_sentiment = "긍정"
    elif node_color == "#BDC3C7":  # 연한 회색 (중립)
        node_sentiment = "중립"
    elif node_color == "#E74C3C":  # 연한 빨간색 (부정)
        node_sentiment = "부정"
    else:
        node_sentiment = "무감정"

    # 빈도수 추출 (Title에서 추출)
    try:
        node_frequency = int(re.search(r"\(빈도: (\d+)\)", node_title).group(1))
    except AttributeError:
        node_frequency = 0  # 빈도수 정보가 없으면 0으로 설정

    # 데이터 추가
    all_nodes_data.append({
        "Node ID": node_id,
        "Node Type": node_type,
        "Label": node_label,
        "Sentiment": node_sentiment,
        "Frequency": node_frequency,
        "Source Nodes": []  # 초기값으로 빈 리스트 설정 (연관 노드일 경우 추가적으로 설정)
    })

# DataFrame으로 변환
all_nodes_df = pd.DataFrame(all_nodes_data)

# 디버깅: 모든 노드 출력
print("[DEBUG] 모든 노드 데이터:")
print(all_nodes_df.head(10))

# 디버깅: 모든 엣지 데이터 출력
print("[DEBUG] 모든 엣지 데이터:")
for edge in net.edges:
    print(f"Source: {edge['from']}, Target: {edge['to']}")

# 엣지 데이터를 활용하여 Source Nodes 설정
for edge in net.edges:
    source = edge["from"]
    target = edge["to"]

    # 디버깅: 현재 처리 중인 엣지
    print(f"[DEBUG] 처리 중인 엣지: Source={source}, Target={target}")

    # 타겟 노드가 연관 노드인 경우 Source Nodes에 소스 노드 추가
    if "_" in target:  # 타겟 노드가 연관 노드
        target_idx = all_nodes_df[all_nodes_df["Node ID"] == target].index
        if not target_idx.empty:
            current_sources = all_nodes_df.at[target_idx[0], "Source Nodes"]
            if isinstance(current_sources, list):  # 리스트 형태인 경우
                current_sources.append(source)
            else:  # 초기값이 없거나 비어 있을 경우
                current_sources = [source]
            # Source Nodes 갱신
            all_nodes_df.at[target_idx[0], "Source Nodes"] = current_sources

# Source Nodes를 문자열로 변환 (콤마로 연결)
all_nodes_df["Source Nodes"] = all_nodes_df["Source Nodes"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# 정렬: 중심 노드 기준, 빈도수 기준 내림차순
all_nodes_df = all_nodes_df.sort_values(by=["Node Type", "Frequency"], ascending=[True, False])

# 결과 저장
output_file = "spring.xlsx"
all_nodes_df.to_excel(output_file, index=False)

print(f"Processed node data saved to '{output_file}'")